# 마이데이터 투자탭 데모 

## Import libraries

In [70]:
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly

import pandas as pd
import numpy as np

from sklearn.preprocessing import RobustScaler, PowerTransformer

import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

import pickle

import re

In [2]:
import FinanceDataReader as fdr
import quantstats as qs

In [3]:
## custom libs

from korquanttools.pricevolume.loader import KRXPriceDM
from korquanttools.pricevolume.utils import DateUtil
from korquanttools.pricevolume.config import PathConfig

In [4]:
CWD = Path('.').resolve()
WOORI_DATA_PATH = CWD / 'woori_data'

## Import dataset

In [5]:
# Global parameters

START = 20140101
END = 20221202

WINDOW = 252 # trading day 기준
# WINDOW = 60 # trading day 기준
# WINDOW = 20 # trading day 기준

In [6]:
# Init data module
pricevolume = KRXPriceDM(START, END)
pricevolume.get_info()


        * DM name: KRX_pricevolume
        * DM description: Basic price-volume data imported from KRX website & NAVER finance. Has KOSPI, KOSDAQ, KONEX stocks.
        * birthday: 20211203
        * DM period: 19990101 ~ 
        * Available data: ['lv1', 'open', 'high', 'low', 'close', 'volume', 'dollarvolume', 'marketcap']
        


### 수익률

In [7]:
## stock return
return_df = pd.read_pickle(PathConfig.cache_path / f"temp_return_{START}_{END}")

In [8]:
holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

tradingdays

DatetimeIndex(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07',
               '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-13',
               '2014-01-14', '2014-01-15',
               ...
               '2022-11-21', '2022-11-22', '2022-11-23', '2022-11-24',
               '2022-11-25', '2022-11-28', '2022-11-29', '2022-11-30',
               '2022-12-01', '2022-12-02'],
              dtype='datetime64[ns]', name='trdDd', length=2190, freq=None)

In [9]:
return_df = return_df.loc[tradingdays, :].copy()

### 수익률 통계량 확인 

- 업종 서브페이지에서 어느 정도를 '보합'으로 잡을지 판단하기 위해 확인

In [10]:
pd.DataFrame(return_df.to_numpy().flatten() * 100).describe()

,0
count,5.081375e+06
mean,5.401327e-02
std,3.832073e+00
min,-9.969191e+01
25%,-1.330377e+00
50%,0.000000e+00
75%,1.129235e+00
max,1.605686e+03


### 시가총액

In [11]:
# Data for 2nd Factor: SMB (Small Minus Big)

marcap_df = pricevolume.get_data("marketcap")

In [12]:
marcap_df = marcap_df.astype(float) # object로 되어있었음

In [13]:
marcap_df = marcap_df.loc[tradingdays, :].copy()

### Grouping

In [14]:
with open('INDUSTRY_NAME2CODE.pickle', 'rb') as handle:
    INDUSTRY_NAME2CODE = pickle.load(handle)

In [15]:
INDUSTRY_CODE2NAME = {v:k for k,v in INDUSTRY_NAME2CODE.items()}

In [16]:
ksic_df = pd.read_excel(WOORI_DATA_PATH / 'ksic_lv1lv2.xlsx', skiprows=2)
ksic_df = ksic_df.ffill()
ksic_df = ksic_df.drop_duplicates()
# ksic_df.to_excel('ksic.xlsx')

KRX 산업분류는 표준산업분류를 기초로 했다고는 하나 굉장히 근본없이 한거같다. 제대로 된 업종 분류로 보이지 않음. 

대분류와 중분류가 섞여있는 모습을 보이고, 서비스업 등은 무엇을 지칭하는 것인지 불분명하다. 

KRX에 설명을 적어놓지도 않았다. 이런... 

그래서 아래와 같이 일단 내가 직접 노가다로 대분류 표준산업분류와 매핑을 시켜봤다. 

 0: '서비스업', -->  전문, 과학 및 기술 서비스업(70~73)

 1: '기타금융', --> 금융 및 보험업(64~66)

 2: '섬유의복', --> 제조업(10~34)

 3: '운수창고업', --> 운수 및 창고업(49~52)

 4: '음식료품', --> 제조업(10~34)

 5: '화학', --> 제조업(10~34)

 6: '철강금속', --> 제조업(10~34)

 7: '유통업', --> 운수 및 창고업(49~52)

 8: '건설업', --> 건설업(41~42)

 9: '증권', --> 금융 및 보험업(64~66)

 10: '전기전자', --> 제조업(10~34)

 11: '의약품', --> 제조업(10~34)

 12: '기계', --> 제조업(10~34)

 13: '종이목재', --> 제조업(10~34)

 14: '통신업', --> 정보통신업(58~63)

 15: '기타제조업', --> 제조업(10~34)

 16: '보험', --> 금융 및 보험업(64~66)

 17: '운수장비', --> 제조업(10~34)

 18: '전기가스업', --> 전기, 가스, 증기 및 공기 조절 공급업(35)

 19: '비금속광물', --> 광업(05~08)

 20: '은행', --> 금융 및 보험업(64~66)

 21: '광업', --> 광업(05~08)

 22: '농업, 임업 및 어업', --> 농업, 임업 및 어업(01~03)

 23: '의료정밀', --> 제조업(10~34)

 24: '반도체', --> 제조업(10~34)

 25: '금속', --> 제조업(10~34)

 26: '기타서비스', --> 전문, 과학 및 기술 서비스업(70~73)

 27: '방송서비스', --> 정보통신업(58~63)

 28: '유통', --> 운수 및 창고업(49~52)

 29: '제약', --> 제조업(10~34)

 30: '통신장비', --> 제조업(10~34)

 31: 'IT부품', --> 제조업(10~34)

 32: '기계·장비', --> 제조업(10~34)

 33: '오락·문화', --> 예술, 스포츠 및 여가관련 서비스업(90~91)

 34: '운송장비·부품', --> 제조업(10~34)

 35: '건설', --> 건설업(41~42)

 36: '전기·가스·수도', --> 전기, 가스, 증기 및 공기 조절 공급업(35)

 37: '소프트웨어', --> 정보통신업(58~63)

 38: '인터넷', --> 정보통신업(58~63)

 39: '디지털컨텐츠', --> 정보통신업(58~63)

 40: '출판·매체복제', --> 정보통신업(58~63)

 41: '금융', --> 금융 및 보험업(64~66)

 42: '운송', --> 운수 및 창고업(49~52)

 43: '통신서비스', --> 정보통신업(58~63)

 44: '일반전기전자', --> 제조업(10~34)

 45: '섬유·의류', --> 제조업(10~34)

 46: '컴퓨터서비스', --> 정보통신업(58~63)

 47: '음식료·담배', --> 제조업(10~34)

 48: '비금속', --> 제조업(10~34)

 49: '종이·목재', --> 제조업(10~34)

 50: '의료·정밀기기', --> 제조업(10~34)

 51: '기타제조', --> 제조업(10~34)

 52: '정보기기', --> 제조업(10~34)

 53: '숙박·음식' --> 숙박 및 음식점업(55~56)

In [17]:
CODE2KSIC_MAPPING = {
    0: '전문, 과학 및 기술 서비스업(70~73)',
    1: '금융 및 보험업(64~66)',
    2: '제조업(10~34)',
    3: '운수 및 창고업(49~52)',
    4: '제조업(10~34)',
    5: '제조업(10~34)',
    6: '제조업(10~34)',
    7: '운수 및 창고업(49~52)',
    8: '건설업(41~42)',
    9: '금융 및 보험업(64~66)',
    10: '제조업(10~34)',
    11: '제조업(10~34)',
    12: '제조업(10~34)',
    13: '제조업(10~34)',
    14: '정보통신업(58~63)',
    15: '제조업(10~34)',
    16: '금융 및 보험업(64~66)',
    17: '제조업(10~34)',
    18: '전기, 가스, 증기 및 공기 조절 공급업(35)',
    19: '광업(05~08)',
    20: '금융 및 보험업(64~66)',
    21: '광업(05~08)',
    22: '농업, 임업 및 어업(01~03)',
    23: '제조업(10~34)',
    24: '제조업(10~34)',
    25: '제조업(10~34)',
    26: '전문, 과학 및 기술 서비스업(70~73)',
    27: '정보통신업(58~63)',
    28: '운수 및 창고업(49~52)',
    29: '제조업(10~34)',
    30: '제조업(10~34)',
    31: '제조업(10~34)',
    32: '제조업(10~34)',
    33: '예술, 스포츠 및 여가관련 서비스업(90~91)',
    34: '제조업(10~34)',
    35: '건설업(41~42)',
    36: '전기, 가스, 증기 및 공기 조절 공급업(35)',
    37: '정보통신업(58~63)',
    38: '정보통신업(58~63)',
    39: '정보통신업(58~63)',
    40: '정보통신업(58~63)',
    41: '금융 및 보험업(64~66)',
    42: '운수 및 창고업(49~52)',
    43: '정보통신업(58~63)',
    44: '제조업(10~34)',
    45: '제조업(10~34)',
    46: '정보통신업(58~63)',
    47: '제조업(10~34)',
    48: '제조업(10~34)',
    49: '제조업(10~34)',
    50: '제조업(10~34)',
    51: '제조업(10~34)',
    52: '제조업(10~34)',
    53: '숙박 및 음식점업(55~56)',
    }

In [18]:
industry_df = pd.read_pickle('krx_industry_df_20140101_20221202.pickle')

In [19]:
with open('sid2name.pkl', 'rb') as p:
    sid2name = pickle.load(p)

## 업종별 수익률

In [20]:
return_df.iloc[-1, :]

ISU_SRT_CD
000020    0.006508
000040    0.000000
000050    0.009009
000060   -0.018672
000070    0.014620
            ...   
440200    0.005063
440790   -0.002513
446070   -0.007899
439410   -0.002525
442130    0.007557
Name: 2022-12-02 00:00:00, Length: 3110, dtype: float64

마지막날은 2022-12-02

In [21]:
# DATE_BEFORE = -1

DATE = '2022-01-27'

In [22]:
last_return_v = return_df.loc[DATE, :]

last_marcap_v = marcap_df.loc[DATE, :]
last_group_v = industry_df.loc[DATE, :]

In [23]:
last_group_v.isna().sum()

630

In [24]:
last_lv1_df = pd.DataFrame(data={
    'ret': last_return_v,
    'marcap': last_marcap_v,
    'group_idx': last_group_v,
})

In [25]:
len(last_lv1_df)

3110

In [26]:
last_lv1_df = last_lv1_df.dropna()
last_lv1_df = last_lv1_df.reset_index()
last_lv1_df

,ISU_SRT_CD,ret,marcap,group_idx
0,000020,-0.054622,3.142290e+11,11.0
1,000040,-0.035387,7.075710e+10,17.0
2,000050,-0.035211,3.755892e+11,7.0
3,000060,-0.011881,6.019188e+12,16.0
4,000070,-0.039954,7.202552e+11,1.0
...,...,...,...,...
2474,400840,-0.002375,1.075200e+10,41.0
2475,404990,-0.011555,2.632724e+11,0.0
2476,102370,-0.077673,3.439036e+11,28.0
2477,353590,0.041237,2.600865e+11,28.0


In [27]:
## 표준산업분류로 mapping, 더 이상 idx가 아님. string(object) 임.

last_lv1_df['group_idx'] = last_lv1_df['group_idx'].apply(lambda g: CODE2KSIC_MAPPING[g])

In [28]:
group_marcap_weight_df = last_lv1_df[['group_idx', 'marcap']].groupby(by='group_idx').sum()
group_marcap_weight_df.reset_index(drop=False, inplace=True)

group_marcap_weight_df.rename(columns={'marcap': 'group_marcap_sum'}, inplace=True)

In [29]:
KSIC2GRPMARCAP = {k:v for k, v in group_marcap_weight_df.values}
KSIC2GRPMARCAP

{'건설업(41~42)': 23159049428417.0,
 '광업(05~08)': 20109542164633.0,
 '금융 및 보험업(64~66)': 281814023199434.0,
 '농업, 임업 및 어업(01~03)': 1425805903280.0,
 '숙박 및 음식점업(55~56)': 1266333636366.0,
 '예술, 스포츠 및 여가관련 서비스업(90~91)': 13211576550390.0,
 '운수 및 창고업(49~52)': 124859511645838.0,
 '전기, 가스, 증기 및 공기 조절 공급업(35)': 19017086774385.0,
 '전문, 과학 및 기술 서비스업(70~73)': 239402944996600.0,
 '정보통신업(58~63)': 87424152265288.0,
 '제조업(10~34)': 1496756266307020.0}

In [30]:
last_lv1_df.loc[:, 'group_marcap_sum'] = last_lv1_df['group_idx'].apply(lambda g_nm: KSIC2GRPMARCAP[g_nm])

In [31]:

last_lv1_df['group_marcap_weight'] = last_lv1_df['marcap'] / last_lv1_df['group_marcap_sum']
last_lv1_df['group_weighted_ret'] = last_lv1_df['ret'] * last_lv1_df['group_marcap_weight']


In [32]:
group_tree_df = last_lv1_df[['group_idx', 'group_weighted_ret', 'marcap']].groupby(by='group_idx').sum()
group_tree_df.reset_index(drop=False, inplace=True)

In [33]:
group_tree_df['group_weighted_ret'] = 100 * group_tree_df['group_weighted_ret']

In [34]:
group_tree_df.columns

Index(['group_idx', 'group_weighted_ret', 'marcap'], dtype='object')

In [35]:
group_tree_df['log_marcap'] = group_tree_df['marcap'].apply(np.log)

In [36]:
scaler = RobustScaler()

group_tree_df['robust_marcap'] = scaler.fit_transform(group_tree_df['marcap'].to_frame())
# group_tree_df['robust_marcap'] = group_tree_df['robust_marcap'] + abs(group_tree_df['robust_marcap'].min())

In [37]:
scaler = PowerTransformer(method='yeo-johnson')

group_tree_df['power_marcap'] = scaler.fit_transform(group_tree_df['marcap'].to_frame())

In [38]:
group_tree_df['sqrt_marcap'] = group_tree_df['marcap'].apply(np.sqrt)

In [39]:
chart_name = '업종별 수익률 현황'

treemap_fig = px.treemap(
    group_tree_df, 
    title=chart_name,
    path=[
        px.Constant('KOSPI & KOSDAQ'),
        'group_idx',
    ],
    # values='sqrt_marcap',
    values='marcap',
    color='group_weighted_ret',
    # hover_data=[],
    # color_discrete_sequence=,
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
)

treemap_fig.update_layout(
    margin=dict(t=50, l=25, b=25),
    width=300+150,
    height=400,
    legend=dict(orientation='h'),
    )
treemap_fig.show()


### 피드백 1: 차라리 네모 크기를 주가 상승률로 하는 것은 어떤지? 

1월 27일 글로벌 '패닉 셀링'...亞증시·美선물 폭락

https://www.wowtv.co.kr/NewsCenter/News/Read?articleId=A202201270178&t=NN

시장이 크게 대 폭락 할 때는 수익률 상위 기준? 아예 그리지도 못한다. 

즉, 주식시장 오올-블루 일 때 네모를 시가총액 아닌 수익률 기준으로 그려버리면 +일 떄는 아예 못그린다. 

In [40]:
chart_name = '업종별 수익률 현황'

group_tree_df_abs = group_tree_df.copy()
group_tree_df_abs['group_weighted_ret'] = group_tree_df_abs['group_weighted_ret'].abs()

treemap_fig = px.treemap(
    group_tree_df_abs, 
    title=chart_name,
    path=[
        px.Constant('KOSPI & KOSDAQ'),
        'group_idx',
    ],
    # values='power_marcap',
    values='group_weighted_ret',
    color='group_weighted_ret',
    # hover_data=[],
    # color_discrete_sequence=,
    color_continuous_scale='RdBu',
    color_continuous_midpoint=0,
)

treemap_fig.update_layout(
    margin=dict(t=50, l=25, b=25),
    width=300+150,
    height=400,
    legend=dict(orientation='h'),
    )
treemap_fig.show()


### 피드백 2: 차라리 네모 크기를 보유 종목 비중으로 하는 것은 어떤지? 

이 경우 애초에 업종이라는 2D 위치 정보가 필요하기 때문에 트리맵 자체가 어울리지 않는다. 

보유 비중이라면 단순히 막대/원 그래프로 표현하는 것이 더 깔끔하고 간결. 

만약 트리맵으로 한다면 유저 개개인의 보유 종목 수/비율에 따라 UI가 가변적으로 변할 수밖에 없다. 

In [41]:
len(group_tree_df['group_idx'].unique())

11

In [42]:
edgecase_stocknames = [
    '삼성전자',
    '우리은행',
    'LG CNS',
    '한화에어로스페이스',
    '현대로템',
    '셀트리온',
    '포스코 ICT',
    '롯데렌탈',
    '원티드랩',
    '두산',
    '대한방직',
    '하이트진로2우B',
    '오리온홀딩스',
    '원풍물산',
    'LX세미콘',
]

edgecase_stockreturns = [
    -0.02,
    0.05,
    -0.07,
    0.15,
    0.1,
    -0.13,
    -0.01,
    0.01,
    0.00,
    0.11,
    0.03,
    -0.05,
    0.11,
    0.03,
    -0.05,
]

edgecase_stockweights = np.array([
    700,
    800,
    100,
    30,
    20,
    60,
    5,
    10,
    50,
    30,
    20,
    10,
    40,
    35,
    20,
])

edgecase_stockweights = edgecase_stockweights / sum(edgecase_stockweights)
edgecase_stockweights

array([0.3626943 , 0.41450777, 0.05181347, 0.01554404, 0.01036269,
       0.03108808, 0.00259067, 0.00518135, 0.02590674, 0.01554404,
       0.01036269, 0.00518135, 0.02072539, 0.01813472, 0.01036269])

In [43]:
edgecase_df = pd.DataFrame(
    {
        'stocknames': edgecase_stocknames,
        'stockreturns': edgecase_stockreturns,
        'stockweights': edgecase_stockweights,
    }
)

In [44]:
chart_name = '종목별 수익률 현황'

treemap_fig = px.treemap(
    edgecase_df, 
    title=chart_name,
    path=[
        px.Constant('KOSPI & KOSDAQ'),
        'stocknames',
    ],
    # values='sqrt_marcap',
    values='stockweights',
    color='stockreturns',
    # hover_data=[],
    # color_discrete_sequence=,
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
)

treemap_fig.update_layout(
    margin=dict(t=50, l=25, b=25),
    width=300+150,
    height=400,
    legend=dict(orientation='h'),
    )
treemap_fig.show()


## FnGuide WICS 기준 grouping 

KRX 산업분류는 통계청의 표준산업분류를 기반으로 하기 때문에 제대로 업종 분류를 보여주지 못한다. 

FnGuide의 WICS 는 GICS를 벤치마킹하여 한국 상황에 잘 맞게 grouping을 잘 나눠놨다.
- Industrial, Consumer Cyclical, Consumer Staples 등과 같이 business/retail, cyclicatl/non-cyclical 등의 기준으로 나눈 것들이 좋다. 
- 한국표준산업분류(10차, 2017년)에선 제조업에 너무 많은 항목이 들어간다. (의료제조업, 식음료제조업, 화학제조업 등등)
- KRX 산업분류는 이걸 기준으로 하면서 대분류와 중분류를 적당히 섞기도 하는 등 

https://www.newspim.com/news/view/20190104000248

### Load & Merge WICS mapping table

In [45]:
# 2023-03-28 현재 listed 된 종목들의 FnGuide WICS 업종분류와 KRX 업종분류와 표준산업분류 업종분류 mapping 

wics_ksic_mapping_df = pd.read_excel(WOORI_DATA_PATH / 'FnGuide_WICS_and_KRX_KSIC.xlsx', skiprows=5)
len(wics_ksic_mapping_df)

2427

In [46]:
wics_ksic_mapping_df.columns

Index(['Symbol', 'Name', 'FnGuide Sector Code', 'FnGuide Sector',
       'FnGuide Industry Group Code', 'FnGuide Industry Group',
       'FnGuide Industry Code', 'FnGuide Industry', '거래소 업종코드', '거래소 업종',
       '거래소 업종코드(세부분류)', '거래소 업종 (세부분류)', '한국표준산업분류10차(대분류)',
       '한국표준산업분류코드10차(대분류)', '한국표준산업분류10차(중분류)', '한국표준산업분류코드10차(중분류)',
       '한국표준산업분류10차(소분류)', '한국표준산업분류코드10차(소분류)', '거래소 업종구분'],
      dtype='object')

In [ ]:
wics_ksic_mapping_df['거래소 업종 (세부분류)'].unique() # nan도 존재

array(['코스피 전기,전자', '코스피 의약품', '코스피 화학', '코스피 운수장비', '코스피 서비스업',
       '코스피 철강및금속', '코스닥 일반전기,전자', '코스피 비금속광물', '코스피 유통업', '코스피 금융업',
       '코스피 보험', '코스피 제조업', '코스피 전기가스업', '금융', '코스피 은행', '도매', '코스닥 IT부품',
       '코스피 통신업', '코스피 기계', '코스피 운수창고', '코스피 음식료품', '코스피 섬유,의복', '코스닥 제약',
       '코스피 건설업', '코스피 증권', '코스닥 디지탈컨텐츠', '코스닥 의료,정밀기기', '코스닥 기계,장비',
       '코스닥 150 선물', '코스닥 화학', nan, '코스피 의료정밀', '코스닥 반도체', '코스닥 방송서비스',
       '코스닥 음식료,담배', '코스닥 종이,목재', '코스닥 소프트웨어', '여행·운송서비스', '코스닥 금속',
       '코스닥 통신장비', '코스닥 150 레버리지 0.5', '코스닥 150 선물 인버스', '코스닥 기타 제조',
       '코스닥 비금속', '코스닥 인터넷', '코스닥 운송장비,부품', '소매', '코스닥 통신서비스',
       '코스닥 출판,매체복제', '코스피 종이,목재', '종합건설', '코스닥 섬유,의류', '전문건설',
       '코스닥 컴퓨터서비스', '금융서비스', '코스닥 정보기기', '자동차판매', '육상운송', '코스닥 150'],
      dtype=object)

In [110]:
sid_list = last_lv1_df['ISU_SRT_CD'].values
sid_list

array(['000020', '000040', '000050', ..., '102370', '353590', '404950'],
      dtype=object)

In [116]:
wics_ksic_mapping_df['Symbol'] = wics_ksic_mapping_df['Symbol'].apply(lambda s: s[1:])

In [151]:
WICS_last_lv1_df = pd.merge(last_lv1_df, wics_ksic_mapping_df, how='left', left_on='ISU_SRT_CD', right_on='Symbol',)
WICS_last_lv1_df.tail()

,ISU_SRT_CD,ret,marcap,group_idx,group_marcap_sum,group_marcap_weight,group_weighted_ret,Symbol,Name,FnGuide Sector Code,...,거래소 업종,거래소 업종코드(세부분류),거래소 업종 (세부분류),한국표준산업분류10차(대분류),한국표준산업분류코드10차(대분류),한국표준산업분류10차(중분류),한국표준산업분류코드10차(중분류),한국표준산업분류10차(소분류),한국표준산업분류코드10차(소분류),거래소 업종구분
2474,400840,-0.002375,1.075200e+10,금융 및 보험업(64~66),2.818140e+14,0.000038,-9.062428e-08,400840,하이제7호스팩,FGSC.40,...,코스닥 금융,I.294,금융서비스,금융 및 보험업,K,금융 및 보험 관련 서비스업,66.0,금융 지원 서비스업,661.0,기타금융업
2475,404990,-0.011555,2.632724e+11,"전문, 과학 및 기술 서비스업(70~73)",2.394029e+14,0.001100,-1.270667e-05,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2476,102370,-0.077673,3.439036e+11,운수 및 창고업(49~52),1.248595e+14,0.002754,-2.139358e-04,102370,케이옥션,FGSC.25,...,코스닥 유통,I.284,도매,도매 및 소매업,G,도매 및 상품 중개업,46.0,상품 중개업,461.0,제조업
2477,353590,0.041237,2.600865e+11,운수 및 창고업(49~52),1.248595e+14,0.002083,8.589828e-05,353590,오토앤,FGSC.25,...,코스닥 유통,I.283,자동차판매,도매 및 소매업,G,자동차 및 부품 판매업,45.0,자동차 부품 및 내장품 판매업,452.0,제조업
2478,404950,-0.011765,1.226400e+10,금융 및 보험업(64~66),2.818140e+14,0.000044,-5.119772e-07,404950,DB금융스팩10호,FGSC.40,...,코스닥 금융,I.294,금융서비스,금융 및 보험업,K,금융업,64.0,기타 금융업,649.0,기타금융업


### WICS grouping treemap

In [152]:
WICS_last_lv1_df.columns

Index(['ISU_SRT_CD', 'ret', 'marcap', 'group_idx', 'group_marcap_sum',
       'group_marcap_weight', 'group_weighted_ret', 'Symbol', 'Name',
       'FnGuide Sector Code', 'FnGuide Sector', 'FnGuide Industry Group Code',
       'FnGuide Industry Group', 'FnGuide Industry Code', 'FnGuide Industry',
       '거래소 업종코드', '거래소 업종', '거래소 업종코드(세부분류)', '거래소 업종 (세부분류)',
       '한국표준산업분류10차(대분류)', '한국표준산업분류코드10차(대분류)', '한국표준산업분류10차(중분류)',
       '한국표준산업분류코드10차(중분류)', '한국표준산업분류10차(소분류)', '한국표준산업분류코드10차(소분류)',
       '거래소 업종구분'],
      dtype='object')

In [255]:
# DATE_BEFORE = -1

DATE = '2022-05-17'

In [256]:
last_return_v = return_df.loc[DATE, :]

last_marcap_v = marcap_df.loc[DATE, :]
last_group_v = industry_df.loc[DATE, :]

In [257]:
last_group_v.isna().sum()

612

In [258]:
last_lv1_df = pd.DataFrame(data={
    'ret': last_return_v,
    'marcap': last_marcap_v,
    'group_idx': last_group_v,
})

In [259]:
len(last_lv1_df)

3110

In [260]:
last_lv1_df = last_lv1_df.dropna()
last_lv1_df = last_lv1_df.reset_index()

In [261]:
WICS_last_lv1_df = pd.merge(last_lv1_df, wics_ksic_mapping_df, how='left', left_on='ISU_SRT_CD', right_on='Symbol',)

#### WICS grouping

In [262]:
WICS_last_lv1_df = WICS_last_lv1_df.dropna()

In [263]:
grouping_colname = 'FnGuide Sector'

In [264]:
group_marcap_weight_df = WICS_last_lv1_df[[grouping_colname, 'marcap']].groupby(by=grouping_colname).sum()
group_marcap_weight_df.reset_index(drop=False, inplace=True)

group_marcap_weight_df.rename(columns={'marcap': 'group_marcap_sum'}, inplace=True)

In [265]:
GRPNAME2GRPMARCAP = {k:v for k, v in group_marcap_weight_df.values}
GRPNAME2GRPMARCAP

{'IT': 988308188040426.0,
 '경기소비재': 278176952312022.0,
 '금융': 198116507401157.0,
 '산업재': 247013350910306.0,
 '소재': 191115561956980.0,
 '에너지': 72484803283560.0,
 '유틸리티': 24356205237850.0,
 '의료': 201498947115805.0,
 '통신서비스': 29345532796820.0,
 '필수소비재': 89535917004064.0}

In [266]:
WICS_last_lv1_df.loc[:, 'group_marcap_sum'] = WICS_last_lv1_df[grouping_colname].apply(lambda g_nm: GRPNAME2GRPMARCAP[g_nm])

In [267]:

WICS_last_lv1_df['group_marcap_weight'] = WICS_last_lv1_df['marcap'] / WICS_last_lv1_df['group_marcap_sum']
WICS_last_lv1_df['group_weighted_ret'] = WICS_last_lv1_df['ret'] * WICS_last_lv1_df['group_marcap_weight']


In [268]:
group_tree_df = WICS_last_lv1_df[[grouping_colname, 'group_weighted_ret', 'marcap']].groupby(by=grouping_colname).sum()
group_tree_df.reset_index(drop=False, inplace=True)

In [269]:
group_tree_df['group_weighted_ret'] = 100 * group_tree_df['group_weighted_ret']

In [270]:
group_tree_df.columns

Index(['FnGuide Sector', 'group_weighted_ret', 'marcap'], dtype='object')

In [271]:
group_tree_df['sqrt_marcap'] = group_tree_df['marcap'].apply(np.sqrt)

In [272]:
chart_name = '업종별 수익률 현황'

# Without sqrt scaling
treemap_fig = px.treemap(
    group_tree_df, 
    title=chart_name,
    path=[
        px.Constant('KOSPI & KOSDAQ'),
        grouping_colname,
    ],
    # values='sqrt_marcap',
    values='marcap',
    color='group_weighted_ret',
    # hover_data=[],
    # color_discrete_sequence=,
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
)

treemap_fig.update_layout(
    margin=dict(t=50, l=25, b=25),
    width=300+150,
    height=400,
    legend=dict(orientation='h'),
    )
treemap_fig.show()


In [273]:
chart_name = '업종별 수익률 현황'

# With sqrt scaling
treemap_fig = px.treemap(
    group_tree_df, 
    title=chart_name,
    path=[
        px.Constant('KOSPI & KOSDAQ'),
        grouping_colname,
    ],
    values='sqrt_marcap',
    # values='marcap',
    color='group_weighted_ret',
    # hover_data=[],
    # color_discrete_sequence=,
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
)

treemap_fig.update_layout(
    margin=dict(t=50, l=25, b=25),
    width=300+150,
    height=400,
    legend=dict(orientation='h'),
    )
treemap_fig.show()
